In [1]:
!pip install PuLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 5.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# New Sample Codes for the Optimization

import pulp
import random

# Number of nodes and full set of station types
num_nodes = 50
station_types = ['eb', 'es', 'ec', 'walk']

# Randomly assign station types to each node
#node_stations = {i: random.sample(full_station_types, random.randint(1, len(full_station_types)))
#                 for i in range(1, num_nodes + 1)}

# Randomly assign station types to each node, ensuring 'walk' and start/end stations are included
node_stations = {i: list(set(['walk'] + random.sample(station_types, random.randint(1, len(station_types) - 1))))
                 for i in range(1, num_nodes + 1)}

node_stations

{1: ['es', 'walk'],
 2: ['es', 'ec', 'walk'],
 3: ['ec', 'walk'],
 4: ['es', 'walk'],
 5: ['es', 'eb', 'walk'],
 6: ['es', 'walk'],
 7: ['ec', 'walk'],
 8: ['es', 'eb', 'walk'],
 9: ['es', 'walk'],
 10: ['es', 'eb', 'walk'],
 11: ['es', 'walk'],
 12: ['es', 'walk'],
 13: ['es', 'eb', 'walk'],
 14: ['ec', 'walk'],
 15: ['es', 'walk'],
 16: ['eb', 'walk'],
 17: ['es', 'ec', 'eb', 'walk'],
 18: ['ec', 'eb', 'walk'],
 19: ['ec', 'walk'],
 20: ['es', 'ec', 'walk'],
 21: ['ec', 'walk'],
 22: ['eb', 'walk'],
 23: ['es', 'walk'],
 24: ['es', 'eb', 'walk'],
 25: ['es', 'walk'],
 26: ['walk'],
 27: ['eb', 'walk'],
 28: ['es', 'ec', 'eb', 'walk'],
 29: ['es', 'eb', 'walk'],
 30: ['es', 'ec', 'walk'],
 31: ['eb', 'walk'],
 32: ['es', 'walk'],
 33: ['es', 'walk'],
 34: ['es', 'eb', 'walk'],
 35: ['es', 'walk'],
 36: ['es', 'ec', 'walk'],
 37: ['walk'],
 38: ['eb', 'walk'],
 39: ['ec', 'eb', 'walk'],
 40: ['es', 'eb', 'walk'],
 41: ['walk'],
 42: ['walk'],
 43: ['es', 'eb', 'walk'],
 44: ['ec', 'eb'

In [3]:
# Specify start and end conditions
start_node, start_station = 1, 'walk'  # Example: Starting from node 1 with station B
end_node, end_station = 30, 'walk'      # Example: Ending at node 2 with station B

# Create a MILP problem
prob = pulp.LpProblem("Minimize_Traversal_Cost", pulp.LpMinimize)

# Decision variables for paths between different nodes and station changes
paths = {(i, j, s): pulp.LpVariable(f"path_{i}_{j}_{s}", 0, 1, pulp.LpBinary)
         for i in range(1, num_nodes + 1)
         for j in range(1, num_nodes + 1)
         for s in station_types if i != j}
station_changes = {(i, s1, s2): pulp.LpVariable(f"station_change_{i}_{s1}_{s2}", 0, 1, pulp.LpBinary)
                   for i in range(1, num_nodes + 1)
                   for s1 in station_types
                   for s2 in station_types if s1 != s2}

# Random weights (costs) for each connection and station changes
costs = {(i, j, s): random.randint(5, 20)
         for i in range(1, num_nodes + 1)
         for j in range(1, num_nodes + 1)
         for s in station_types if i != j}

station_change_costs = {(i, s1, s2): 0.1*random.randint(1, 5)
                        for i in range(1, num_nodes + 1)
                        for s1 in station_types
                        for s2 in station_types if s1 != s2}

# constant cost
#station_change_costs = {(i, s1, s2): 0.1
#                        for i in range(1, num_nodes + 1)
#                        for s1 in station_types
#                        for s2 in station_types if s1 != s2}

In [4]:
#station_change_costs

In [5]:
# Objective Function
prob += pulp.lpSum([paths[i, j, s] * costs[i, j, s] for i, j, s in paths]) + \
        pulp.lpSum([station_changes[i, s1, s2] * station_change_costs[i, s1, s2] for i, s1, s2 in station_changes])

# Constraints
for i in range(1, num_nodes + 1):
    for s in station_types:
        # Flow balance for each node and station type
        incoming_flow = pulp.lpSum([paths[j, i, s] for j in range(1, num_nodes + 1) if j != i])
        outgoing_flow = pulp.lpSum([paths[i, j, s] for j in range(1, num_nodes + 1) if j != i])

        # Allow station changes within the node
        incoming_flow += pulp.lpSum([station_changes[i, s2, s] for s2 in station_types if s2 != s])
        outgoing_flow += pulp.lpSum([station_changes[i, s, s2] for s2 in station_types if s2 != s])

        # Start node should only have outgoing flow
        if i == start_node and s == start_station:
            prob += outgoing_flow == 1
            prob += incoming_flow == 0
        # End node should only have incoming flow
        elif i == end_node and s == end_station:
            prob += incoming_flow == 1
            prob += outgoing_flow == 0
        # Intermediate nodes should have equal incoming and outgoing flows
        else:
            prob += incoming_flow == outgoing_flow

# Solve the problem
prob.solve()

# Output results
if pulp.LpStatus[prob.status] == 'Optimal':
    print("Total Cost: ", pulp.value(prob.objective))
    for i, j, s in paths:
        if pulp.value(paths[i, j, s]) == 1:
            print(f"Path from {i} to {j} with station {s} selected. Cost: {costs[i, j, s]}")
    for i, s1, s2 in station_changes:
        if pulp.value(station_changes[i, s1, s2]) == 1:
            print(f"Station change at node {i} from {s1} to {s2}. Cost: {station_change_costs[i, s1, s2]}")
else:
    print("No optimal solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dingyue/PycharmProjects/pythonProject/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/f3/gd2h6jg568xdh7606tr291nm0000gn/T/1aba7d75cdc142de87ae90b19938e432-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f3/gd2h6jg568xdh7606tr291nm0000gn/T/1aba7d75cdc142de87ae90b19938e432-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 207 COLUMNS
At line 52208 RHS
At line 52411 BOUNDS
At line 62812 ENDATA
Problem MODEL has 202 rows, 10400 columns and 20800 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5 - 0.01 seconds
Cgl0002I 103 variables fixed
Cgl0004I processed model has 200 rows, 10297 columns (10297 integer (10297 of which binary)) and 20594 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0038I Initial state - 0 integers unsa